# Reed-Solomon 

First, the original all-zero bitstream $u(x) = u_0 + u_1x + \cdots + u_{n-1}x^{n-1}$ is generated and it gets corrupted with a random error $e(x) = e_0 + e_1x + \cdots + e_{n-1}x^{n-1}$. The received bitstream is split in symbols of m bits to create the $r(x) = u(x) + e(x) = r_0 + r_1x + \cdots + r_{n-1}x^{n-1}$ polynomial. The syndrom $S$ is then calculated using the formula $S_i = r(\alpha^i)$. We assume that there are $\nu$ errors at positions $j_1, j_2, \cdots, j_{\nu}$ of the polynomial $e(x)$. Let $\beta_i = \alpha^{j_i}$ and t the number of erroneous symbols that can be corrected, so that the syndroms are calculated as:

$\begin{array}
\text S_1 = r(\alpha) = e_{j_1}\beta_1 + e_{j_2}\beta_2 + \cdots + e_{j_{\nu}}\beta_{\nu} \\
\text S_2 = r(\alpha^2) = e_{j_1}{\beta_1}^2 + e_{j_2}{\beta_2}^2 + \cdots + e_{j_{\nu}}{\beta_{\nu}}^2 \\
\vdots \\
\text S_{2t} = r(\alpha^{2t}) = e_{j_1}{\beta_1}^{2t} + e_{j_2}{\beta_2}^{2t} + \cdots + e_{j_{\nu}}{\beta_{\nu}}^{2t} \\
\end{array}$

The polynomial $\sigma(x)$ is computed using an autoregressive model. We make a matrix from the syndroms, where the first t syndroms are used to predict the next syndrome. The matrix is contructed as shown below:

$\begin{bmatrix} 
S_1 & S_2 & \cdots & S_{t-1} & S_t \\
S_2 & S_3 & \cdots & S_t & S_{t+1} \\
&& \vdots \\
S_{t-1} & S_t & \cdots & S_{2t-3} & S_{2t-2} \\
S_t & S_{t+1} & \cdots & S_{2t-2} & S_{2t-1} \\
\end{bmatrix} 
 * 
\begin{bmatrix}
\sigma_t\\
\sigma_{t-1}\\
\vdots\\
\sigma_2\\
\sigma_1\\
\end{bmatrix}
 = 
\begin{bmatrix}
-S_{t+1}\\
-S_{t+2}\\
\vdots\\
-S_{2t-1}\\
-S_{2t}\\
\end{bmatrix}$

Then the value of the errors $e_{j_1}$ can be calculated by solving any t of the equatations written above:

$\begin{bmatrix} 
\beta_1 & \beta_2 & \cdots & \beta_{\nu-1} & \beta_{\nu} \\
{\beta_1}^2 & {\beta_2}^2 & \cdots & {\beta_{\nu-1}}^2 & {\beta_{\nu}}^2 \\
&& \vdots \\
{\beta_1}^t & {\beta_2}^t & \cdots & {\beta_{\nu-1}}^t & {\beta_{\nu}}^t \\
\end{bmatrix} 
 * 
\begin{bmatrix}
e_{j_1}\\
e_{j_2}\\
\vdots\\
e_{j_t}\\
\end{bmatrix}
 = 
\begin{bmatrix}
S_1\\
S_2\\
\vdots\\
S_t\\
\end{bmatrix}$

Finally, the original message can be retreived by adding the errors to the received signal

In [15]:
#Atsc reed-sololon (207,187) can correct up to 10 consequtive corrupted bits
import _pickle as cPickle


In [1]:


def gf_add(x, y):
    return x ^ y

def gf_sub(x, y):
    return x ^ y # in binary galois field, subtraction is just the same as addition (since we mod 2)

def gf_mult_noLUT(x, y, prim=0):
    '''Multiplication in Galois Fields without using a precomputed look-up table (and thus it's slower)
    by using the standard carry-less multiplication + modular reduction using an irreducible prime polynomial'''

    ### Define bitwise carry-less operations as inner functions ###
    def cl_mult(x,y):
        '''Bitwise carry-less multiplication on integers'''
        z = 0
        i = 0
        while (y>>i) > 0:
            if y & (1<<i):
                z ^= x<<i
            i += 1
        return z

    def bit_length(n):
        '''Compute the position of the most significant bit (1) of an integer. Equivalent to int.bit_length()'''
        bits = 0
        while n >> bits: bits += 1
        return bits

    def cl_div(dividend, divisor=None):
        '''Bitwise carry-less long division on integers and returns the remainder'''
        # Compute the position of the most significant bit for each integers
        dl1 = bit_length(dividend)
        dl2 = bit_length(divisor)
        # If the dividend is smaller than the divisor, just exit
        if dl1 < dl2:
            return dividend
        # Else, align the most significant 1 of the divisor to the most significant 1 of the dividend (by shifting the divisor)
        for i in range(dl1-dl2,-1,-1):
            # Check that the dividend is divisible (useless for the first iteration but important for the next ones)
            if dividend & (1 << i+dl2-1):
                # If divisible, then shift the divisor to align the most significant bits and XOR (carry-less subtraction)
                dividend ^= divisor << i
        return dividend
    
    ### Main GF multiplication routine ###

    # Multiply the gf numbers
    result = cl_mult(x,y)
    # Then do a modular reduction (ie, remainder from the division) with an irreducible primitive polynomial so that it stays inside GF bounds
    if prim > 0:
        result = cl_div(result, prim)

    return result

def init_tables(m, prim):
    '''Precompute the logarithm and anti-log tables for faster computation later, using the provided primitive polynomial.'''
    # prim is the primitive (binary) polynomial. Since it's a polynomial in the binary sense,
    # it's only in fact a single galois field value between 0 and 255, and not a list of gf values.
    global gf_length
    global gf_exp, gf_log
    gf_length = 2**m
    gf_exp = [0] * gf_length # anti-log (exponential) table
    gf_log = [0] * gf_length # log table
    # For each possible value in the galois field 2^8, we will pre-compute the logarithm and anti-logarithm (exponential) of this value
    x = 1
    for i in range(0, gf_length-1):
        gf_exp[i] = x # compute anti-log for this value and store it in a table
        gf_log[x] = i # compute log at the same time
        x = gf_mult_noLUT(x, 2, prim)

        # If you use only generator==2 or a power of 2, you can use the following which is faster than gf_mult_noLUT():
        #x <<= 1 # multiply by 2 (change 1 by another number y to multiply by a power of 2^y)
        #if x & 0x100: # similar to x >= 256, but a lot faster (because 0x100 == 256)
            #x ^= prim # substract the primary polynomial to the current value (instead of 255, so that we get a unique set made of coprime numbers), this is the core of the tables generation

    # Optimization: double the size of the anti-log table so that we don't need to mod 255 to
    # stay inside the bounds (because we will mainly use this table for the multiplication of two GF numbers, no more).
#     for i in range(255, 512):
#         gf_exp[i] = gf_exp[i - 255]
    return [gf_log, gf_exp]

def gf_mul(x,y):
    if x==0 or y==0:
        return 0
    return gf_exp[(gf_log[x] + gf_log[y]) % (gf_length-1)]

def gf_div(x,y):
    if y==0:
        raise ZeroDivisionError()
    if x==0:
        return 0
    return gf_exp[(gf_log[x] + (gf_length-1) - gf_log[y]) % (gf_length-1)]

def gf_pow(x, power):
    return gf_exp[(gf_log[x] * power) % (gf_length-1)]

def gf_inverse(x):
    return gf_exp[(gf_length-1) - gf_log[x]] # gf_inverse(x) == gf_div(1, x)

def gf_poly_scale(p,x):
    r = [0] * len(p)
    for i in range(0, len(p)):
        r[i] = gf_mul(p[i], x)
    return r

def gf_poly_add(p,q):
    r = [0] * max(len(p),len(q))
    for i in range(0,len(p)):
        r[i+len(r)-len(p)] = p[i]
    for i in range(0,len(q)):
        r[i+len(r)-len(q)] ^= q[i]
    return r

def gf_poly_mul(p,q):
    '''Multiply two polynomials, inside Galois Field'''
    # Pre-allocate the result array
    r = [0] * (len(p)+len(q)-1)
    # Compute the polynomial multiplication (just like the outer product of two vectors,
    # we multiply each coefficients of p with all coefficients of q)
    for j in range(0, len(q)):
        for i in range(0, len(p)):
            r[i+j] ^= gf_mul(p[i], q[j]) # equivalent to: r[i + j] = gf_add(r[i+j], gf_mul(p[i], q[j]))
                                                         # -- you can see it's your usual polynomial multiplication
    return r

def gf_poly_eval(poly, x):
    '''Evaluates a polynomial in GF(2^p) given the value for x. This is based on Horner's scheme for maximum efficiency.'''
    y = poly[0]
    for i in range(1, len(poly)):
        y = gf_mul(y, x) ^ poly[i]
    return y


# In[10]:


def rwh_primes1(n):
    # http://stackoverflow.com/questions/2068372/fastest-way-to-list-all-primes-below-n-in-python/3035188#3035188
    ''' Returns  a list of primes < n '''
    sieve = [True] * (n/2)
    for i in range(3,int(n**0.5)+1,2):
        if sieve[i/2]:
            sieve[i*i/2::i] = [False] * ((n-i*i-1)/(2*i)+1)
    return [2] + [2*i+1 for i in range(1,n/2) if sieve[i]]

def find_prime_polys(generator=2, c_exp=8, fast_primes=False, single=False):
    '''Compute the list of prime polynomials for the given generator and galois field characteristic exponent.'''
    # fast_primes will output less results but will be significantly faster.
    # single will output the first prime polynomial found, so if all you want is to just find one prime polynomial to generate the LUT for Reed-Solomon to work, then just use that.

    # A prime polynomial (necessarily irreducible) is necessary to reduce the multiplications in the Galois Field, so as to avoid overflows.
    # Why do we need a "prime polynomial"? Can't we just reduce modulo 255 (for GF(2^8) for example)? Because we need the values to be unique.
    # For example: if the generator (alpha) = 2 and c_exp = 8 (GF(2^8) == GF(256)), then the generated Galois Field (0, 1, a, a^1, a^2, ..., a^(p-1)) will be galois field it becomes 0, 1, 2, 4, 8, 16, etc. However, upon reaching 128, the next value will be doubled (ie, next power of 2), which will give 256. Then we must reduce, because we have overflowed above the maximum value of 255. But, if we modulo 255, this will generate 256 == 1. Then 2, 4, 8, 16, etc. giving us a repeating pattern of numbers. This is very bad, as it's then not anymore a bijection (ie, a non-zero value doesn't have a unique index). That's why we can't just modulo 255, but we need another number above 255, which is called the prime polynomial.
    # Why so much hassle? Because we are using precomputed look-up tables for multiplication: instead of multiplying a*b, we precompute alpha^a, alpha^b and alpha^(a+b), so that we can just use our lookup table at alpha^(a+b) and get our result. But just like in our original field we had 0,1,2,...,p-1 distinct unique values, in our "LUT" field using alpha we must have unique distinct values (we don't care that they are different from the original field as long as they are unique and distinct). That's why we need to avoid duplicated values, and to avoid duplicated values we need to use a prime irreducible polynomial.

    # Here is implemented a bruteforce approach to find all these prime polynomials, by generating every possible prime polynomials (ie, every integers between field_charac+1 and field_charac*2), and then we build the whole Galois Field, and we reject the candidate prime polynomial if it duplicates even one value or if it generates a value above field_charac (ie, cause an overflow).
    # Note that this algorithm is slow if the field is too big (above 12), because it's an exhaustive search algorithm. There are probabilistic approaches, and almost surely prime approaches, but there is no determistic polynomial time algorithm to find irreducible monic polynomials. More info can be found at: http://people.mpi-inf.mpg.de/~csaha/lectures/lec9.pdf
    # Another faster algorithm may be found at Adleman, Leonard M., and Hendrik W. Lenstra. "Finding irreducible polynomials over finite fields." Proceedings of the eighteenth annual ACM symposium on Theory of computing. ACM, 1986.

    # Prepare the finite field characteristic (2^p - 1), this also represent the maximum possible value in this field
    root_charac = 2 # we're in GF(2)
    field_charac = int(root_charac**c_exp - 1)
    field_charac_next = int(root_charac**(c_exp+1) - 1)

    prim_candidates = []
    if fast_primes:
        prim_candidates = rwh_primes1(field_charac_next) # generate maybe prime polynomials and check later if they really are irreducible
        prim_candidates = [x for x in prim_candidates if x > field_charac] # filter out too small primes
    else:
        prim_candidates = range(field_charac+2, field_charac_next, root_charac) # try each possible prime polynomial, but skip even numbers (because divisible by 2 so necessarily not irreducible)

    # Start of the main loop
    correct_primes = []
    for prim in prim_candidates: # try potential candidates primitive irreducible polys
        seen = [0] * (field_charac+1) # memory variable to indicate if a value was already generated in the field (value at index x is set to 1) or not (set to 0 by default)
        conflict = False # flag to know if there was at least one conflict

        # Second loop, build the whole Galois Field
        x = 1
        for i in range(field_charac):
            # Compute the next value in the field (ie, the next power of alpha/generator)
            x = gf_mult_noLUT(x, generator, prim)

            # Rejection criterion: if the value overflowed (above field_charac) or is a duplicate of a previously generated power of alpha, then we reject this polynomial (not prime)
            if x > field_charac or seen[x] == 1:
                conflict = True
                break
            # Else we flag this value as seen (to maybe detect future duplicates), and we continue onto the next power of alpha
            else:
                seen[x] = 1

        # End of the second loop: if there's no conflict (no overflow nor duplicated value), this is a prime polynomial!
        if not conflict: 
            correct_primes.append(prim)
            if single: return prim

    # Return the list of all prime polynomials
    return correct_primes # you can use the following to print the hexadecimal representation of each prime polynomial: print [hex(i) for i in correct_primes]


In [2]:



import numpy as np


# In[2]:


class GaloisField:
        
    def add(self,x, y):
        return x ^ y

    def sub(self,x, y):
        # in binary galois field, subtraction is just the same as addition (since we mod 2)
        return x ^ y 
    
    def mul(self,x, y):
        if x==0 or y==0:
            return 0
        return self.exp[(self.log[x] + self.log[y]) % (self.length-1)]

    def div(self,x,y):
        if y==0:
            raise ZeroDivisionError()
        if x==0:
            return 0
        return self.exp[(self.log[x] + (self.length-1) - self.log[y]) % (self.length-1)]

    def gf_pow(self, x, power):
        return self.exp[(self.log[x] * power) % (self.length-1)]
    
    def gf_inverse(self, x):
        return self.exp[(self.length-1) - self.log[x]]
    
    def conv(self,a,b):
        if (len(b) > len(a)):
            temp = a
            a = b
            b = temp

        a_pad = [*([0]*(len(b)-1)), *a, *([0]*(len(b)-1))]
        b_rev = list(reversed(b))

        res = [0]*(len(a_pad)-1)

        for i in range(0,len(a_pad)-len(b)+1):
            partial_sum = 0
            for j in range(0,len(b)):
                partial_sum = self.add(partial_sum,self.mul(a_pad[i+j],b[j]))
            res[i] = partial_sum

        return res
    
    def init_tables(self):
        
        '''Precompute the logarithm and anti-log tables for faster computation later, 
           using the provided primitive polynomial.'''
        # prim is the primitive (binary) polynomial. Since it's a polynomial in the binary sense,
        # it's only in fact a single galois field value between 0 and 255, and not a list of gf values.
        
        m = self.order
        prim = self.prim
        gf_length = self.length
        gf_exp = [0] * gf_length # anti-log (exponential) table
        gf_log = [0] * gf_length # log table
        # For each possible value in the galois field 2^8, 
        # we will pre-compute the logarithm and anti-logarithm (exponential) of this value
        x = 1
        for i in range(0, gf_length-1):
            gf_exp[i] = x # compute anti-log for this value and store it in a table
            gf_log[x] = i # compute log at the same time

            x <<= 1 # multiply by 2 (change 1 by another number y to multiply by a power of 2^y)
            if x >= gf_length:
                # substract the primary polynomial to the current value 
                # (instead of Galois Field length, so that we get a unique set made of coprime numbers), 
                # this is the core of the tables generation
                x ^= prim 
                
        return [gf_log, gf_exp]
    
    def det(self, matrix, mul = 1):
        '''Recursively computes the determinant of a square matrix in Galois Field'''
        
        width = len(matrix)
        if width == 1:
            return self.mul(mul, matrix[0][0])
        else:
            total = 0
            for i in range(width):
                m = []
                for j in range(1, width):
                    buff = []
                    for k in range(width):
                        if k != i:
                            buff.append(matrix[j][k])
                    m.append(buff)
                total = self.add(total, self.mul(mul, self.det(m, matrix[0][i])))
            return total
    
    def solveLinearSystem(self, matrix, array):
        '''Solves a linear system in Galois Field using Cramer's rule and 
           returns the answer to array ans[]''' 
        
        ans = []
        for i in range(0, len(matrix)):
            
            # We need to use deepcopy because temp = matrix returns a pointer to matrix
            # and all changes to temp also affect matrix
            #temp = deepcopy(matrix)
            temp = cPickle.loads(cPickle.dumps(matrix, -1))
            
            # We iteratively substitute each column of the matrix with the array
            # and the compute its determinant in the Galois Field
            for j in range(0, len(array)):
                temp[j][i] = array[j]

            ans.append(self.div(S.det(temp), self.det(matrix)))
        return ans
    
                
    def __init__(self, x, m, d):
        self.element = x
        self.order = m
        self.prim = d
        self.length = 2**m
        [self.log, self.exp] = self.init_tables()


# In[3]:

#parameters for ATSC 
t = 10 # maximum number of erroneous symbols that can be corrected
m = 8 # order of Galois Field 
n = 2**m -1 # length of Galois Field
d = find_prime_polys(2,m,single=True)  
a = GaloisField(2,m,d) # primitive element of Galois Field


# In[4]:


# Compute the generator polynomial of the Reed Solomon code using convolution
# The first coefficient corresponds to x^(2t)
p = 1
g = GaloisField([1, a.gf_pow(a.element, p)], m, d)
for i in range(p+1,p+2*t):
    g.element = g.conv(g.element, [1, a.gf_pow(a.element, i)])
    
print("Generator polynomial of Reed Solomon code with m = {} and t = {}:\n{}".format(m,t,list(reversed(g.element))))


# In[5]:


field_el = GaloisField([],m,d)
for j in range(0,n):
    field_el.element.append(a.gf_pow(a.element,j))
    
print("The non zero elements of GF(2^{}) are:".format(m))
for j in range(0,n):
    print ("a^{} = {}".format(j,field_el.element[j]))


# In[6]:


# Find which elements a^i are included in the generator polynomial
power = np.zeros(len(g.element),dtype=int)
for i in range(0,len(g.element)):
    for j in range(0,n):
        if (g.element[i] == field_el.element[j]):
            power[i] = j
            
print("Exponents of a to which each coefficient of the generator polynomial corresponds to:\n{}".format(list(reversed(power))))


# In[ ]:





Generator polynomial of Reed Solomon code with m = 8 and t = 10:
[2, 95, 50, 1, 113, 112, 86, 7, 225, 194, 106, 137, 243, 223, 244, 201, 13, 120, 127, 91, 162]
The non zero elements of GF(2^8) are:
a^0 = 1
a^1 = 2
a^2 = 4
a^3 = 8
a^4 = 16
a^5 = 32
a^6 = 64
a^7 = 128
a^8 = 29
a^9 = 58
a^10 = 116
a^11 = 232
a^12 = 205
a^13 = 135
a^14 = 19
a^15 = 38
a^16 = 76
a^17 = 152
a^18 = 45
a^19 = 90
a^20 = 180
a^21 = 117
a^22 = 234
a^23 = 201
a^24 = 143
a^25 = 3
a^26 = 6
a^27 = 12
a^28 = 24
a^29 = 48
a^30 = 96
a^31 = 192
a^32 = 157
a^33 = 39
a^34 = 78
a^35 = 156
a^36 = 37
a^37 = 74
a^38 = 148
a^39 = 53
a^40 = 106
a^41 = 212
a^42 = 181
a^43 = 119
a^44 = 238
a^45 = 193
a^46 = 159
a^47 = 35
a^48 = 70
a^49 = 140
a^50 = 5
a^51 = 10
a^52 = 20
a^53 = 40
a^54 = 80
a^55 = 160
a^56 = 93
a^57 = 186
a^58 = 105
a^59 = 210
a^60 = 185
a^61 = 111
a^62 = 222
a^63 = 161
a^64 = 95
a^65 = 190
a^66 = 97
a^67 = 194
a^68 = 153
a^69 = 47
a^70 = 94
a^71 = 188
a^72 = 101
a^73 = 202
a^74 = 137
a^75 = 15
a^76 = 30
a^77 = 60
a

In [20]:
import numpy as np
from copy import deepcopy
import time


start_time = time.time()


t = 10 # maximum number of erroneous symbols that can be corrected
m = 8 # order of the Galois Field

n = 2**m - 1
d = find_prime_polys(2,m,single=True)

print("Prime polynomial of GF(2^{}) in binary form: {}".format(m, bin(d)))

# The first non-zero, non-one element of the Galois Field
a = GaloisField(2,m,d)


# In[4]:


# Generate an all-zero codeword of length n*m bits and deliberate corrupt some bits, 
# whose locations are specified in e_pos array

rb = np.zeros(m*n, dtype = int)

# Damage a number of random bits
err_num = 10 # Caution: If more than t symbols get damaged, Reed Solomon decoding fails 
e_pos = np.random.randint(0, n*m, err_num, dtype=int)
print("Position of the corrupted bits: ", e_pos)

# Else corrupt specific bits (for debugging purposes)
# e_pos = np.array([12, 13, 15, 27, 48, 49])
# e_pos = np.array([12,13,14,25,26,52,54])

for i in range(0,len(e_pos)):
    rb[e_pos[i]] = not(rb[e_pos[i]])

# Split the message in symbols of length m bits
rm = np.zeros(len(rb)//m, dtype=int)
for i in range(0,len(rb)//m):
    for j in range (0,m):
        rm[i] = (rm[i] << 1) + rb[i*m+j]

print("The corrupted message symbols in decimal form are: ",rm)
r = GaloisField(rm,m,d)


# In[5]:


# Calculate syndroms in array s
# s[i] = r(a^i) for i = 1...2t where r(x) the currupted message in polynomial form

s = GaloisField([],m,d)
for i in range(0,2*t):
    s.element.append(r.element[n-1])
    for j in range(0, n-1):
        temp = GaloisField(2,m,d)
        temp.element = a.gf_pow(a.element,i+1)
        temp.element = temp.gf_pow(temp.element, j+1)
        s.element[i] = s.add(s.element[i],temp.mul(temp.element, r.element[n-j-2]))
    
print("The syndrom of the message is:", s.element)


# In[6]:


# Compute sigma polynomial using auto-regression method
# by solving the linear system S*sigma = cons_s

S = GaloisField(np.zeros([t, t], dtype=int),m,d)
cons_s = GaloisField(np.zeros(t, dtype=int),m,d)
for i in range(0,t):
    for j in range(0,t):
        S.element[i][j] = s.element[i+j]
    cons_s.element[i] = s.element[t+i]

if (S.det(S.element) == 0):
    print("The syndrom cannot be defined, because det(S) == 0")
else:
    sigma = S.solveLinearSystem(S.element, cons_s.element)

    print("S\t\t sigma\t cons_s")
    for i in range(0, len(S.element)):
        print(S.element[i][:], "\t", sigma[i], "\t", cons_s.element[i])


# In[7]:


# Find roots of sigma polynomial
s_roots_exp = []
for i in range(0, n-1):
    # Compute the value of polynomial sigma where x -> a^i for i = 0...n-1
    ai = a.gf_pow(a.element, i+1)
    sr = GaloisField(1, m, d)
    
    # Construct polynomial from coefficients stored in array sigma[]
    # (the highest order coefficient is stored in sigma[0])
    for j in range(0, t):
        temp = a.mul(sigma[j],a.gf_pow(ai,t-j))
        sr.element = sr.add(sr.element,temp)
    
    if (sr.element == 0):
        # s_roots_exp contains the powers of a for which sigma(a^i) == 0
        s_roots_exp.append(i+1)
        
print(s_roots_exp)


# In[8]:


# The error positions can be found by inversing the roots of sigma polynomial

b = []
err_pos = []
for i in range(0, len(s_roots_exp)):
    root = a.gf_pow(a.element, s_roots_exp[i])
    b.append(a.gf_inverse(root))
    
    # Find error positions form most significant bit
    err_pos.append(n - a.log[b[i]])
    
print("Symbols with errors are: ", err_pos)



# In[9]:


# Having found the erroneous symbols, we need to determine the value of each error

B = GaloisField([],m,d)
cons_b = GaloisField([],m,d)
for i in range(0,t):
    
    for j in range(0,t):
        B.element.append(B.gf_pow(b[j],i+1))

    cons_b.element.append(s.element[i])

B.element = np.reshape(B.element, [t, len(B.element)//t]) #must be here?

err = B.solveLinearSystem(B.element, cons_b.element)

print("B\t\t err\t cons_b")

for i in range(0, len(S.element)):
    print(B.element[i][:], "\t", err[i], "\t", cons_b.element[i])


# In[10]:


# The corrupted can be fixed by subtracting the error values of err array
# from the erroneous symbols as specified in err_pos array

corrected = GaloisField([], m, d)


#corrected.element = deepcopy(r.element)
corrected.element = cPickle.loads(cPickle.dumps(r.element, -1))
for i in range(0, len(err)):
    corrected.element[err_pos[i]-1] = corrected.add(r.element[err_pos[i]-1], err[i])
    
print("Corrupted message:", r.element)
print("Corrected message:", corrected.element)
print("%s seconds (sloww)" % (time.time() - start_time))




# In[ ]:





Prime polynomial of GF(2^8) in binary form: 0b100011101
Position of the corrupted bits:  [ 623 1451 1337 1090 1757 1603  360  614 1899  952]
The corrupted message symbols in decimal form are:  [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0 128   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   2   1   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0 128   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0  32   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0  64   0   0   0   0   0   0   0   0   0   0   0   0
   0  16   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0

In [ ]:
# Το σημείο εύρεσης και διόρθωσης των εσφαλμένων τιμών του πολυωνυμου είναι το πιο χρονοβορο λογω της χρησης deepcopy για την αντιγραφη των πινακων 
# και η αναγνώρισή τους όταν υπάρχουν  πολλα loops καθιστα αρκετα χρονοβορα την εκτελεση.
#Για το λόγο αυτο χρησιμοποίηση στη θέση του το cPickle που είναι λιγο πιο γρήγορο. (μονο 1 λεπτο γρηγορτοτερο)

In [7]:
#Συγκριτικά
#https://stackoverflow.com/questions/10128351/any-alternative-to-a-very-slow-deepcopy-in-a-dfs